In [0]:
import sys
sys.path.append('../')

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

In [0]:
import requests 
import pandas as pd
import urllib.parse
import json
import time
from datetime import date
import math

from common import *

In [0]:
since, datestr = get_max_value_from_last_file("companies", "properties.hs_lastmodifieddate.timestamp")


if json_exists('companies', datestr):
   raise ValueError('data for today already exist. If you want to proceed, comment these lines')

# token=dbutils.secrets.get()

print(datestr, since)

In [0]:
##Get all company properties (custom and default) and load into string used in URL
property_string = get_all_properties('companies/properties', 'properties', token)
print(property_string[:1000])

In [0]:
#Get all created or modified companies
company_list=[]
url= f'https://api.hubapi.com/companies/v2/companies/recent/modified?'
headers = {
  'content-type': 'application/json',
  'authorization': f'Bearer {token}'
}

file_count = 0
hasMore = True

parameter_dict = {'since': since, 'offset': 0, 'count':100}

while hasMore:
    parameters = urllib.parse.urlencode(parameter_dict)
    get_url = url + parameters + property_string
    
    try:
        r = request_with_retry(get_url, headers)
    
    except Exception as e:
        if len(company_list) > 0:
            file_name=str(file_count).zfill(5)
            save_new_json(company_list, 'companies', datestr, file_name)
            file_count +=1
            contact_list = []

    response_dict = json.loads(r.text)
    print(response_dict["total"])  
        
    response_dict = json.loads(r.text)
    
    timestamps = [record['properties']['hs_lastmodifieddate']['timestamp'] for record in response_dict['results']]
    if len(timestamps) > 0:
        print(min(timestamps), max(timestamps))
    
    company_list.extend(response_dict['results'])
    hasMore=response_dict['hasMore']
    parameter_dict['offset'] = response_dict['offset']
    
    print(f'{response_dict["total"]=}, {response_dict["offset"]=} {parameter_dict["since"]=}')
    
    if len(company_list)>= 10000:  #10000 partial saves
        company_list = []
        file_name = str(file_count).zfill(5)
        save_new_json(company_list, 'companies', datestr, file_name)
        file_count += 1

    if response_dict['hasMore']==False or response_dict["offset"] == 10000:
        file_name=str(file_count).zfill(5)
        save_new_json(company_list, 'companies', datestr, file_name)
        
        if response_dict["offset"] >= 10000:
            warnings.warn('Offset 10000 reached, run me more often!')
            delete_folder_with_current_incremental_data('companies', datestr, dbutils)

            #running full load
            target_notebook_path = "load_full"
            result = dbutils.notebook.run(target_notebook_path, 0, {"datestr": datestr})

        break
        
print("You've succesfully parsed through {} contact records and added them to a list".format(len(company_list)))

In [0]:
company_list[:50]

In [0]:
company_list[-50:]